<h2 align='center'>Codebasics ML Course: ML Flow Dagshub Tutorial</h2>

In [11]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings

In [12]:
warnings.filterwarnings('ignore')

In [13]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100]))

In [14]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

#### Handle class imbalance

In [15]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)
np.unique(y_train_res, return_counts=True)

(array([0, 1]), array([619, 619]))

### Track Experiments

In [16]:
# models list of sets
models = [
    (
        "Logistic Regression", # model name
        {"C": 1, "solver": 'liblinear'}, # params
        LogisticRegression(), # model
        (X_train, y_train), # train set
        (X_test, y_test) # test set
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    ( 
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    ( # with balance dataset
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [17]:
# Initialize an empty list to store classification reports for each model
reports = []

# Loop over a list of models with their names, parameters, and datasets
for model_name, params, model, train_set, test_set in models:
    
    # Unpack training and test data
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    # Set the model's hyperparameters
    model.set_params(**params)
    
    # Train the model on the training data
    model.fit(X_train, y_train)
    
    # Predict the labels on the test set
    y_pred = model.predict(X_test)
    
    # Generate classification report (precision, recall, f1-score, etc.)
    # and convert it to a dictionary
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # Append the report to the list
    reports.append(report)


In [18]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [19]:
# dagshub setup

import dagshub
dagshub.init(repo_owner='dhirajpatra', repo_name='mlflow_dagshub_demo', mlflow=True)

Initialized MLflow to track repo "dhirajpatra/mlflow_dagshub_demo"

Repository dhirajpatra/mlflow_dagshub_demo initialized!

In [20]:
# Load environment variables from a .env file (only needed if you've previously set DAGsHub credentials)
from dotenv import load_dotenv
load_dotenv()

import os

# Fetch DAGsHub credentials and MLflow tracking URI from environment variables
MLFLOW_TRACKING_USERNAME = os.getenv('MLFLOW_TRACKING_USERNAME')
MLFLOW_TRACKING_PASSWORD = os.getenv('MLFLOW_TRACKING_PASSWORD')
MLFLOW_TRACKING_URI = os.getenv('MLFLOW_TRACKING_URI')  # Example: https://dagshub.com/username/repo.mlflow

# Set the experiment name for MLflow; it groups related runs together
mlflow.set_experiment("Anomaly Detection")


2025/06/26 14:22:24 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detection' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/6cc5104a2b77464980937b6c74e9585e', creation_time=1750927945029, experiment_id='0', last_update_time=1750927945029, lifecycle_stage='active', name='Anomaly Detection', tags={}>

In [21]:
# Optional: use this line if you want to switch to local MLflow tracking instead of DAGsHub
mlflow.set_tracking_uri("http://localhost:5000")


In [22]:
# Loop over each trained model and its corresponding classification report
for i, element in enumerate(models):
    model_name = element[0]   # Name of the model (e.g., "RandomForest")
    params = element[1]       # Hyperparameters used for training the model
    model = element[2]        # The actual trained model object
    report = reports[i]       # Classification report for this model
    
    # Start a new MLflow run for logging
    with mlflow.start_run(run_name=model_name):        
        # Log model hyperparameters
        mlflow.log_params(params)

        # Log key evaluation metrics from the classification report
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  

        # Log the model itself using the appropriate MLflow flavor
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")

2025/06/26 14:22:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/06/26 14:22:57 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/26 14:22:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/0/runs/f2ad5bf3ef0043198bb9a665eacdfa64
🧪 View experiment at: http://localhost:5000/#/experiments/0


2025/06/26 14:22:59 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/26 14:22:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/0/runs/03bcd3b4183f46cba757846f0bef1025
🧪 View experiment at: http://localhost:5000/#/experiments/0


2025/06/26 14:23:02 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
2025/06/26 14:23:02 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/0/runs/faf678f1eb8f4abaa827bc52d1493606
🧪 View experiment at: http://localhost:5000/#/experiments/0


2025/06/26 14:23:05 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/0/runs/a6698c8d39dd454291a55cbb6c151137
🧪 View experiment at: http://localhost:5000/#/experiments/0
